In [12]:
!pip install kfp==1.8.14

In [13]:
import kfp
from kfp import dsl
from kfp.components import create_component_from_func, InputPath, OutputPath
from kubernetes import client as k8s

In [14]:
client = kfp.Client()

In [15]:
client.list_experiments()

{'experiments': [{'created_at': datetime.datetime(2023, 11, 7, 3, 51, 16, tzinfo=tzlocal()),
                  'description': '231107_Park_Test',
                  'id': 'acbdfaad-d14a-46ef-82fc-2f5715f8d1c7',
                  'name': '231107_Park_Test',
                  'resource_references': [{'key': {'id': 'modeltest',
                                                   'type': 'NAMESPACE'},
                                           'name': None,
                                           'relationship': 'OWNER'}],
                  'storage_state': 'STORAGESTATE_AVAILABLE'},
                 {'created_at': datetime.datetime(2023, 11, 8, 20, 10, 59, tzinfo=tzlocal()),
                  'description': 'first experiment',
                  'id': '498323bc-fe83-44a4-bf7b-4215abcd539e',
                  'name': 'lab0',
                  'resource_references': [{'key': {'id': 'modeltest',
                                                   'type': 'NAMESPACE'},
                        

In [21]:
def getData(bucket: str):
    import pandas as pd
    import influxdb_client
    import os
    
    config_path = "/mnt/data/config.ini"
    
    retDf = pd.DataFrame()
    colName = []
    client = influxdb_client.InfluxDBClient.from_config_file(config_path)
    query_api = client.query_api()
    query = f'from(bucket:"{bucket}")\
    |> range(start:-1d)\
    |> filter(fn:(r)=> r._measurement=="SensorData" and r.Name=="M16M")'

    result = query_api.query(query=query)

    for table in result:
        col = table.records[0].get_field()
        colName.append(col)
        temp = []
        for record in table.records:
            temp.append((int(record.get_value())))
        retDf[col] = temp
    
    retDf = retDf[["MM", "DD", "Day", "HH", "Min", "Sec",
    "Illuminance", "Movement", "Manual", "On",
     "Brightness"]]
    
    print(retDf)

def okData(train_data):
    import pandas as pd
    
    print(train_data["MM"])

BASE_IMAGE = "python:3.7"
getData_op = create_component_from_func(getData, 
                                    base_image=BASE_IMAGE,
                                    packages_to_install=["pandas", "influxdb_client"])

okData_op = create_component_from_func(okData, 
                                    base_image=BASE_IMAGE,
                                    packages_to_install=["pandas", "influxdb_client"])


# pod setting
def pod_defaults(op):
    volume_name = "a305-storages"
    volume_mount_path = "/mnt/data"
    
    op.set_memory_request('10Mi').set_cpu_request('10m')
    
    volume = k8s.V1Volume(
        name=volume_name,
        persistent_volume_claim=k8s.V1PersistentVolumeClaimVolumeSource(claim_name=volume_name),
    )

    volume_mount = k8s.V1VolumeMount(
        mount_path=volume_mount_path,
        name=volume_name,
    )

    op.add_pvolumes({volume_mount_path: volume})

    return op

In [24]:
# 2. Build a pipeline using the component                      

# pipeline의 이름은 Calculation pipeline
@dsl.pipeline(
   name='Calculation pipeline2',
   description='A toy pipeline that performs arithmetic calculations.'
)
def calc_pipeline(
   bucket: str =''
):
    add_task = pod_defaults(getData_op(bucket))  # 1st task

In [25]:
arguments = {"bucket" : "MaiL"}
EXPERIMENT_NAME = 'get-data-pipeline-1'
kfp.Client().create_run_from_pipeline_func(calc_pipeline,
                                           arguments=arguments,
                                           experiment_name=EXPERIMENT_NAME)

RunPipelineResult(run_id=e335d4a5-9e79-4479-a580-2f3f6a885732)